# Building Machine Learning Systems That Don't Suck


This notebook creates a [SageMaker Pipeline](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_building_pipeline.html) to build an end-to-end Machine Learning system to solve the problem of classifying penguin species. With a SageMaker Pipeline, you can create, automate, and manage end-to-end Machine Learning workflows at scale.

You can find more information about Amazon SageMaker in the [Amazon SageMaker Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html). The [AWS Machine Learning Blog](https://aws.amazon.com/blogs/machine-learning/) is an excellent source to stay up-to-date with SageMaker.

This example uses the [Penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data).

<img src='images/penguins.png' alt='Penguins' width="800">

This notebook is part of the [Machine Learning School](https://www.ml.school) program.


## Session 1 - Introduction and Initial Setup

The machine learning system we'll build during this program consists of four main pipelines: A **training** pipeline, an **inference** pipeline, a **deployment** pipeline, and a **monitoring** pipeline.

Here is an architectural diagram showing how the system is structured:

<a href="images/diagram.png" target="_blank"> <img src="images/diagram.png" alt="SageMaker architectural diagram of the system" style="max-width: 740px;" /></a>

Throughout the sessions, we'll build each of these pipelines. We'll also build variations to show you different alternatives and best practices.

Let's start by setting up the environment and preparing to run the notebook.


We can run this notebook in [Local Mode](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-local-mode.html) to test some of the system components in your local environment. Unfortunately, not every component is supported in Local Mode.

Setting the `LOCAL_MODE` variable to `True` will run every supported pipeline component locally. Setting the variable to `False` will run the pipeline in SageMaker.


Let's now load the environment variables we need to run the notebook.


If you are running the pipeline in Local Mode on an ARM64 machine (for example, on Apple Silicon), you will need to use a custom Docker image to train and evaluate the model. Let's create a variable indicating if we are running on an ARM64 machine.


Let's create a configuration dictionary with different settings depending on whether we are running the pipeline in Local Mode. We'll use this dictionary to configure the pipeline components.


Let's now initialize a few variables that we'll need throughout the notebook:


## Session 2 - Exploratory Data Analysis

Let's run Exploratory Data Analysis on the [Penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data). The goal of this session is to understand the data and the problem we are trying to solve.


Let's load the Penguins dataset:


We can see the dataset contains the following columns:

1. `species`: The species of a penguin. This is the column we want to predict.
2. `island`: The island where the penguin was found
3. `culmen_length_mm`: The length of the penguin's culmen (bill) in millimeters
4. `culmen_depth_mm`: The depth of the penguin's culmen in millimeters
5. `flipper_length_mm`: The length of the penguin's flipper in millimeters
6. `body_mass_g`: The body mass of the penguin in grams
7. `sex`: The sex of the penguin

If you are curious, here is the description of a penguin's culmen:

<img src='images/culmen.jpeg' alt='Culmen' width="600">


Now, let's get the summary statistics for the features in our dataset.


Let's now display the distribution of values for the three categorical columns in our data:


The distribution of the categories in our data are:

-   `species`: There are 3 species of penguins in the dataset: Adelie (`152`), Gentoo (`124`), and Chinstrap (`68`).
-   `island`: Penguins are from 3 islands: Biscoe (`168`), Dream (`124`), and Torgersen (`52`).
-   `sex`: We have `168` male penguins, `165` female penguins, and `1` penguin with an ambiguous gender (`.`).

Let's replace the ambiguous value in the `sex` column with a `null` value:


Next, let's check for any missing values in the dataset.


Let's get rid of the missing values. For now, we are going to replace the missing values with the most frequent value in the column. Later, we'll use a different strategy to replace missing numeric values.


Let's visualize the distribution of categorical features.


Let's visualize the distribution of numerical columns.


Let's display the covariance matrix of the dataset. The "covariance" measures how changes in one variable are associated with changes in a second variable. In other words, the covariance measures the degree to which two variables are linearly associated.


Here are three examples of what we get from interpreting the covariance matrix below:

1. The positive covariance of 50.26 between culmen length and flippler length suggests that larger values of culmen length are associated with larger values of flipper length. As one increases, generally so does the other.
2. The positive covariance of 2596.97 between culmen length and body mass suggests that heavier penguins generally have longer culmens. There is a tendency for these two variables to increase together.
3. The negative covariance of -742.66 between culmen depth and body mass suggests a general tendency that penguins with deeper culmens weigh less.


Let's now display the correlation matrix. "Correlation" measures both the strength and direction of the linear relationship between two variables:


Here are three examples of what we get from interpreting the correlation matrix below:

1. Penguins that weight more tend to have longer flippers.
2. Penguins with a shallower culmen tend to have longer flippers.
3. Penguins with longer culmens tend to have longer flippers.


Let's display the distribution of species by island:


Let's display the distribution of species by sex.


## Session 3 - Splitting and Transforming the Data

In this session we'll build a simple [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with one step to split and transform the data:

<a href="images/processing-step.png" target="_blank"> <img src="images/processing-step.png" alt="High-level overview of the Preprocessing Step" style="max-width: 740px;" /></a>

We'll use a [Scikit-Learn Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) for the transformations, and a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing) with a [SKLearnProcessor](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#scikit-learn-processor) to execute a preprocessing script. Check the [SageMaker Pipelines Overview](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) for an introduction to the fundamental components of a SageMaker Pipeline.


### Step 1 - Creating the Preprocessing Script

The first step we need in the pipeline is a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing) to run a script that will split and transform the data.

This Processing Step will create a SageMaker Processing Job in the background, run the script, and upload the output to S3. You can use Processing Jobs to perform data preprocessing, post-processing, feature engineering, data validation, and model evaluation. Check the [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) SageMaker's SDK documentation for more information.


We will store the script in a folder called `processing` and add it to the system path so we can later import it as a module.


Let's now create the script:


Let's test the script to ensure everything is working as expected:


### Step 2 - Caching Configuration

Several SageMaker Pipeline steps support caching. When a step runs, and dependending on the configured caching policy, SageMaker will try to reuse the result of a previous successful run of the same step. You can find more information about this topic in [Caching Pipeline Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-caching.html).

Let's define a caching policy that we'll reuse on every step:


### Step 3 - Pipeline Configuration

We can parameterize a SageMaker Pipeline to make it more flexible. In this case, we'll use a parameter to pass the location of the dataset we want to process. We can execute the pipeline with different datasets by changing the value of this parameter. Check [Pipeline Parameters](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html) for more information.


### Step 4 - Setting up the Processing Step

Let's now define the [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) that we'll use in the pipeline to run the script that will split and transform the data.

A processor gives the Processing Step information about the hardware and software that SageMaker should use to launch a Processing Job. To run the script we created, we need access to Scikit-Learn, so we can use the [SKLearnProcessor](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#scikit-learn-processor) processor that comes out-of-the-box with the SageMaker's Python SDK.

SageMaker manages the infrastructure of a Processing Job. It provisions resources for the duration of the job, and cleans up when it completes. The Processing Container image that SageMaker uses to run a Processing Job can either be a SageMaker built-in image or a custom image:

<a href="images/processing-job.png" target="_blank"> <img src="images/processing-job.png" alt="High-level overview of a SageMaker Processing Job" style="max-width: 740px;" /></a>

The [Data Processing with Framework Processors](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job-frameworks.html) page discusses other built-in processors you can use. The [Docker Registry Paths and Example Code](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html) page contains information about the available framework versions for each region.


Let's now define the Processing Step that we'll use in the pipeline.

This step will specify the list of inputs that we'll access from the preprocessing script. In this case, the input is the dataset we stored in S3. We also have a few outputs that we want SageMaker to capture when the Processing Job finishes.


### Step 5 - Creating the Pipeline

We can now create the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


## Session 4 - Training the Model

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with a [step to train a model](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training). Check [Train a Model with TensorFlow](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#train-a-model-with-tensorflow) for more information about training a model with TensorFlow.

<a href="images/training-step.png" target="_blank"> <img src="images/training-step.png" alt="High-level overview of the Training Step" style="max-width: 740px;" /></a>

We'll also introduce experiment tracking using [Amazon SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) and [Comet](https://www.comet.com/site/?utm_source=svpino_course&utm_medium=partner&utm_content=github).


### Step 1 - Creating the Training Script

Let's create the training script. This script is responsible for training a neural network using the train data, validating the model, and saving it so we can later use it.


We will store the script in a folder called `training` and add it to the system path so we can later import it as a module.


We can now create the script inside the folder:


Let's test the script to ensure everything is working as expected:


### Step 2 - Setting up the Training Step

We can now create a [Training Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) that we can add to the pipeline. This Training Step will create a SageMaker Training Job in the background, run the training script, and upload the output to S3. Check the [TrainingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TrainingStep) SageMaker's SDK documentation for more information.

SageMaker manages the infrastructure of a Training Job. It provisions resources for the duration of the job, and cleans up when it completes. The Training Container image that SageMaker uses to run a Training Job can either be a SageMaker built-in image or a custom image.

<a href="images/training-job.png" target="_blank"> <img src="images/training-job.png" alt="High-level overview of a SageMaker Training Job" style="max-width: 740px;" /></a>

The [Available Deep Learning Container Images](https://github.com/aws/deep-learning-containers/blob/master/available_images.md) page contains the list of available containers for each region.


Our training script uses [Comet](https://www.comet.com/site/?utm_source=svpino_course&utm_medium=partner&utm_content=github) to track metrics from the Training Job. We need to create a `requirements.txt` file to install the Comet library in the training container.


SageMaker uses the concept of an [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) to handle end-to-end training and deployment tasks. For this example, we will use the built-in [TensorFlow Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-estimator) to run the training script we wrote before.

Notice the list of hyperparameters defined below. SageMaker will pass these hyperparameters as arguments to the entry point of the training script.

We are going to use [Comet](https://www.comet.com/site/?utm_source=svpino_course&utm_medium=partner&utm_content=github) and [SageMaker Experiments](https://sagemaker.readthedocs.io/en/v2.174.0/experiments/sagemaker.experiments.html) to track metrics from the Training Job. SageMaker Experiments will use the list of metric definitions to decide which metrics to track and how to parse them from the Training Job logs. For more information, check [Manage Machine Learning with Amazon SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) and the [SageMaker Experiments' SDK documentation](https://sagemaker.readthedocs.io/en/v2.174.0/experiments/sagemaker.experiments.html).

Here are the environment variables we need to set on the traininng container:

-   `COMET_API_KEY`: This is your [Comet](https://www.comet.com/site/?utm_source=svpino_course&utm_medium=partner&utm_content=github) API key.
-   `COMET_PROJECT_NAME`: The name of the project where you want to track the experiments.


We can now create a [Training Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training). This Training Step will create a SageMaker Training Job in the background, run the training script, and upload the output to S3. Check the [TrainingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TrainingStep) SageMaker's SDK documentation for more information.

This step will receive the train and validation split from the preprocessing step as inputs.

Here, we are using three input channels, `train`, `validation`, and `pipeline`. SageMaker will automatically create an environment variable corresponding to each of these channels following the format `SM_CHANNEL_[channel_name]`:

-   `SM_CHANNEL_TRAIN`: This environment variable will contain the path to the training data coming from the preprocessing step.
-   `SM_CHANNEL_VALIDATION`: This environment variable will contain the path to the validation data coming from the preprocessing step.
-   `SM_CHANNEL_PIPELINE`: This environment variable will contain the path to the transformation pipeline that we saved in the preprocessing step.

Notice that we are creating a function that we can later reuse to create a training step using a different estimator.


### Step 3 - Creating the Pipeline

Let's define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


## Session 5 - Custom Training Container

This session creates a custom Docker image to train the model and have full control of the environment where the training script runs.

For this example, we'll run the training script using [Keras 3](https://keras.io) with a [JAX](https://jax.readthedocs.io/en/latest/index.html) backend. Check [Adapting your own Docker container to work with SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers-adapt-your-own.html) for more information about using your own Docker containers.


### Step 1 - Preparing the Docker Image

The first step is to copy the training script to a folder where we'll prepare the Docker image. We are going to reuse the training script we created before, since it's compatible with the latest version of Keras.


Since we are creating a new Docker image, we need to install the libraries we need in the training container. We'll use a `requirements.txt` file to install these libraries. Notice that we are installing `jax` to run it as our backend.

The `sagemaker-training` library contains the common functionality necessary to create a container compatible with SageMaker and its Python SDK.


We can now create the Dockerfile containing the instructions to build the training image. Notice how this image will automatically run the `train.py` script when it starts.

To use JAX as the backend for our model, we need to set the `KERAS_BACKEND` environment variable to `jax`.


### Step 2 - Building the Docker Image

We can now build the Docker image using the `docker build` command. We are going to define the name of this image using the `IMAGE_NAME` variable.


### Step 3 - Pushing Docker Image to ECR

We can now push the Docker image to Amazon Elastic Container Registry (ECR). This is a fully-managed Docker container registry where we can manage Docker container images. This step is necessary to make the image available to SageMaker when running the pipeline.


### Step 4 - Setting up the Training Step

Let's now compute the name of the training image we'll use to run the Training Job.

If we are running in `LOCAL_MODE`, we'll use the name of the image we built before (`IMAGE_NAME`). Otherwise, we'll use the name of the image we pushed to ECR.


We can now create an [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) and a [Training Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) using the function we created before.


### Step 5 - Creating the Pipeline

Let's define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


## Session 6 - Tuning the Model

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with a [step to tune the model](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning) using a Hyperparameter Tuning Job.

<a href="images/tuning-step.png" target="_blank"> <img src="images/tuning-step.png" alt="High-level overview of the Tuning Step" style="max-width: 740px;" /></a>


### Step 1 - Enabling Tuning

Since we could use the Training of the Tuning Step to create a model, we'll define a constant to indicate which approach we want to run. Notice that the Tuning Step is not supported in Local Mode.


### Step 2 - Setting up a Tuning Step

Let's now create a [Tuning Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning). This Tuning Step will create a SageMaker Hyperparameter Tuning Job in the background and use the training script to train different model variants and choose the best one. Check the [TuningStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep) SageMaker's SDK documentation for more information.

The Tuning Step requires a [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) reference to configure the Hyperparameter Tuning Job.

Here is the configuration that we'll use to find the best model:

1. `objective_metric_name`: This is the name of the metric the tuner will use to determine the best model.
2. `objective_type`: This is the objective of the tuner. It specifies whether it should minimize the metric or maximize it. In this example, since we are using the validation accuracy of the model, we want the objective to be "Maximize." If we were using the loss of the model, we would set the objective to "Minimize."
3. `metric_definitions`: Defines how the tuner will determine the metric's value by looking at the output logs of the training process.

The tuner expects the list of the hyperparameters you want to explore. You can use subclasses of the [Parameter](https://sagemaker.readthedocs.io/en/stable/api/training/parameter.html#sagemaker.parameter.ParameterRange) class to specify different types of hyperparameters. This example explores different values for the `epochs` hyperparameter.

Finally, you can control the number of jobs and how many of them will run in parallel using the following two arguments:

-   `max_jobs`: Defines the maximum total number of training jobs to start for the hyperparameter tuning job.
-   `max_parallel_jobs`: Defines the maximum number of parallel training jobs to start.


We can now create the Tuning Step using the tuner we configured before. SageMaker will create a Hyperparameter Tuning Job in the background and use the training script to train different model variants and choose the best one.

<a href="images/tuning-job.png" target="_blank"> <img src="images/tuning-job.png" alt="High-level overview of SageMaker's Hyperparameter Tuning Jobs" style="max-width: 740px;" /></a>


### Step 3 - Creating the Pipeline

Let's define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


## Session 7 - Evaluating the Model

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with a step to evaluate the model using the holdout set we created during the preprocessing step.

<a href="images/evaluation-step.png" target="_blank"> <img src="images/evaluation-step.png" alt="High-level overview of the Evaluation Step" style="max-width: 740px;" /></a>


### Step 1 - Creating the Evaluation Script

We'll use a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing) to execute the evaluation script.

This script is responsible for loading the model we created and evaluating it on the test set. Before finishing, this script will generate an evaluation report of the model.


We will store the script in a folder called `evaluation` and add it to the system path so we can later import it as a module.


We can now create the script inside the folder:


Let's test the script to ensure everything is working as expected:


### Step 2 - Referencing the Model Assets

One of the inputs to the evaluation step is the model coming from the Training or the Tuning step. We can use the `USE_TUNING_STEP` flag to determine whether we created the model using a Training Step or a Tuning Step.

In case we are using the Tuning Step, we can use the [TuningStep.get_top_model_s3_uri()](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep.get_top_model_s3_uri) function to get the model assets from the top performing training job of the Hyperparameter Tuning Job.


### Step 3 - Mapping the Output to a Property File

SageMaker supports mapping outputs from a Processing Step to property files. This is useful when we want to access a specific property from the pipeline.

We'll map the evaluation report to a property file. Check [How to Build and Manage Property Files](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-propertyfile.html) for more information.


### Step 4 - Setting up the Evaluation Step

To run the evaluation script, we will use a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing) configured with a [TensorFlowProcessor](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job-frameworks-tensorflow.html) because the script needs access to TensorFlow.


We are now ready to define the [Processing Step](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) that will run the evaluation script:


### Step 5 - Creating the Pipeline

Let's define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


## Session 8 - Registering the Model

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with a step to [register the model](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-version.html) in the [SageMaker Model Registry](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry.html).

<a href="images/registration-step.png" target="_blank"> <img src="images/registration-step.png" alt="High-level overview of the Registration Step" style="max-width: 740px;" /></a>


### Step 1 - Configuring the Model Package Group

First, let's define the name of the group where we'll register the model. The Model Registry uses groups to organize the versions of a model:


### Step 2 - Creating the Model

Let's now create the model that we'll register in the Model Registry. The model we trained uses TensorFlow, so we can use the built-in [TensorFlowModel](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-serving-model) class to create an instance of the model:


### Step 3 - Configuring Model Metrics

When we register a model in the Model Registry, we can attach relevant metadata to it. We'll use the evaluation report we generated during the evaluation step to populate the [metrics](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_metrics.ModelMetrics) of this model:


### Step 4 - Registering the Model

We can use a [Model Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-model) to register the model. Check the [ModelStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.model_step.ModelStep) SageMaker's SDK documentation for more information.


### Step 5 - Creating the Pipeline

Let's define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


## Session 9 - Conditional Registration

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with a condition to register the model only if its accuracy is above a predefined threshold.

Here's a high-level overview of the Condition Step:

<a href="images/condition-step.png" target="_blank"> <img src="images/condition-step.png" alt="High-level overview of the Condition Step" style="max-width: 740px;" /></a>


### Step 1 - Configuring the Accuracy Threshold

Let's define a new [Pipeline Parameter](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html) to specify the minimum accuracy that the model should reach for it to be registered.


### Step 2 - Setting up a Fail Step

If the model's accuracy is not greater than or equal to our threshold, we will send the pipeline to a [Fail Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-fail) with the appropriate error message. Check the [FailStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.fail_step.FailStep) SageMaker's SDK documentation for more information.


### Step 3 - Defining the Condition

We can use a [ConditionGreaterThanOrEqualTo](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.conditions.ConditionGreaterThanOrEqualTo) condition to compare the model's accuracy with the threshold. Look at the [Conditions](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_building_pipeline.html#conditions) section in the documentation for more information about the types of supported conditions.


### Step 4 - Setting up the Condition Step

Let's now use a [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) together with the evaluation report we generated to determine whether the model's accuracy is above the threshold:


### Step 5 - Creating the Pipeline

We can now define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


## Session 10 - Serving the Model

This session builds a simple [Flask](https://flask.palletsprojects.com/) application to serve the model.

<a href="images/deploying-flask.png" target="_blank"> <img src="images/deploying-flask.png" alt="High-level overview of deploying a model using a Flask wrapper" style="max-width: 740px;" /></a>

Keep in mind that, while good for development and testing, this is not the best approach for production systems.


### Step 1 - Retrieving List of Approved Models

We want to serve the latest approved model from the Model Registry. We can use the [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html) API to get this model:


### Step 2 - Downloading the Model

Let's now download the model assets from the location specified in the Model Registry to your local environment.


We will store this model in a folder called `serving`:


Let's now download the model assets into the folder:


### Step 3 - Creating the Serving Script

Let's now write a simple Flask script to serve the model.

When this application receives the first request, it will unpack and load the model into memory. From there, it will use the model to make predictions on the incoming requests.


### Step 4 - Running the Flask Application

We can now run the Flask application to serve the model from a terminal using the following command:

```bash
$ flask --app program/code/serving/app.py --debug run --host=0.0.0.0 --port=4242
```

After the server is running, you can send a POST request to the server to get a prediction. Here is an example using the `curl` command:

```bash
$ curl --location --request POST 'http://localhost:4242/predict' \
    --header 'Content-Type: text/plain' \
    --data-raw '0.6569590202313976, -1.0813829646495108, 1.2097102831892812, 0.9226343641317372, 1.0, 0.0, 0.0'
```


## Session 11 - Deploying the Model

This session deploys the model from the Model Registry to an endpoint. We'll do it manually, using the SageMaker SDK. Check [Deploy to a SageMaker Endpoint](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#deploy-to-a-sagemaker-endpoint) for more information about deploying a model to an endpoint.

<a href="images/deploying-model.png" target="_blank"> <img src="images/deploying-model.png" alt="High-level overview of deploying a model to a SageMaker endpoint" style="max-width: 740px;" /></a>


### Step 1 - Configuring the Endpoint Name

Let's start by defining the name of the endpoint where we'll deploy the model:


### Step 2 - Creating a Model Package

To deploy a model using the SageMaker's Python SDK, we need to create a [Model Package](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.ModelPackage) using the ARN of the model from the Model Registry. Remember we got the ARN of the latest approved model in the previous section.


### Step 3 - Deploying the Model

Let's now deploy the model to an endpoint.


#| hide

<div class="alert" style="background-color:#0066cc;"><strong>Note:</strong> 
    The <code>%%script</code> cell magic is a convenient way to prevent the notebook from executing a specific cell. If you want to run the cell, comment out the line containing the <code>%%script</code> cell magic.
</div>


### Step 4 - Testing the Endpoint

After deploying the model, we can test the endpoint to make sure it works.

Each line of the payload we'll send to the endpoint contains the information of a penguin. Notice the model expects data that's already transformed. We can't provide the original data from our dataset because the model we registered will not work with it.

The endpoint will return the predictions for each of these lines.


Let's send the payload to the endpoint and print its response:


## Session 12 - Deploying From the Pipeline

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with a step to automatically deploy the model to an endpoint.

We'll use a [Lambda Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-lambda) to create an endpoint and deploy the model.

Here's a high-level overview of the Deploy Step:

<a href="images/deploy-step.png" target="_blank"> <img src="images/deploy-step.png" alt="High-level overview of the Deploy Step" style="max-width: 740px;" /></a>


### Step 1 - Configuring Data Capture Settings

We want to enable [Data Capture](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-capture.html) as part of the endpoint configuration. With Data Capture we can record the inputs and outputs of the endpoint to use them later for monitoring the model. We need to configuration settings to enable Data Capture:

-   `DATA_CAPTURE_PERCENTAGE`: Represents the percentage of traffic that we want to capture.
-   `DATA_CAPTURE_DESTINATION`: Specifies the S3 location where we want to store the captured data.


### Step 2 - Setting up the Lambda Function

Let's start by writing a Lambda function that takes the model information and deploys it to an endpoint.

There are three components that make up a SageMaker endpoint:

<a href="images/endpoint.png" target="_blank"> <img src="images/endpoint.png" alt="An overview of the three components of an Endpoint" style="max-width: 740px;" /></a>


We'll store the code of the function in a folder called `lambda`:


Let's now write the code of the function:


### Step 3 - Setting up Lambda Permissions

We need to ensure our Lambda function has permission to interact with SageMaker, so let's create a new role with the appropriate permissions.


### Step 4 - Creating the Lambda Function

Let's now create the Lambda function in AWS. We'll pass the configuration settings we defined before as environment variables to the Lambda function.


### Step 5 - Setting up the Lambda Step

We can now define the [Lambda Step](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.lambda_step.LambdaStep) that will run the function to deploy the model. We'll do this in a function that we can reuse later.

This step will send the model package ARN we want to deploy to the Lambda function as an input parameter.


### Step 6 - Modifying the Condition Step

We need to modify the [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) to include the new deployment step. If the condition succeeds, we will register and deploy the model.


### Step 7 - Creating the Pipeline

We can now define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


### Step 8 - Testing the Endpoint

Let's test the endpoint to make sure it works.

The `wait_for_endpoint` function will wait until the endpoint is ready to receive requests.


## Session 13 - Deploying From an Event

This session modifies the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) to register the model with `PendingManualApproval` status and deploys it whenever its status changes to `Approved`.

<a href="images/deploying-from-event.png" target="_blank"> <img src="images/deploying-from-event.png" alt="High-level overview of deploying a model using EventBridge" style="max-width: 740px;" /></a>

We will use [Amazon EventBridge](https://aws.amazon.com/pm/eventbridge/) to trigger a Lambda function that will deploy the model whenever its status changes from "PendingManualApproval" to "Approved."


### Step 1 - Configuring the Model Package Group

We need to define the name of a new group where we'll register models with `PendingManualApproval` status.


### Step 2 - Setting Up EventBridge

We can now create an EventBridge rule that triggers the deployment process whenever a model approval status becomes `Approved`. To do this, let's define the event pattern that will trigger the deployment process. Check [Model package state change](https://docs.aws.amazon.com/sagemaker/latest/dg/automating-sagemaker-with-eventbridge.html#eventbridge-model-package) for more information.


Let's now create the EventBridge rule:


Now, we need to define the target of the rule. The target will trigger whenever the rule matches an event. In this case, we want to trigger the Lambda function we created before:


### Step 3 - Configuring the Lambda Permissions

Finally, we need to give the Lambda function permissions to be triggered by the EventBridge rule:


### Step 4 - Registering the Model

We need to modify the [Model Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-model) to register the model using `PendingManualApproval` status.


### Step 5 - Modifying the Condition Step

Let's modify the [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) to include the new registration step. If the condition succeeds, we will register the model with `PendingManualApproval` status.


### Step 6 - Creating the Pipeline

Let's define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


## Session 14 - Building an Inference Pipeline

This session creates an inference pipeline to control the data that goes in and comes out of the endpoint.

Deploying the model we trained directly to an endpoint doesn't lets us control the data that goes in and comes out of the endpoint. The TensorFlow model we trained requires transformed data, which makes it useless to other applications:

<a href="images/basic-model.png" target="_blank"> <img src="images/basic-model.png" alt="Basic Model" style="max-width: 740px;" /></a>

To fix this, we can create an Inference Pipeline using SageMaker to control the data that goes in and comes out of the endpoint.

Our inference pipeline will have three components:

1. A preprocessing component that will transform the input data into the format the model expects.
2. The TensorFlow model.
3. A postprocessing component that will transform the output of the model into a human-readable format.

<a href="images/inference-pipeline.png" target="_blank"> <img src="images/inference-pipeline.png" alt="Inference pipeline" style="max-width: 740px;" /></a>

We want our endpoint to handle unprocessed data in CSV and JSON format and return the penguin's species. Here is an example of the payload input we want the endpoint to support:

```{json}
{
    "island": "Biscoe",
    "culmen_length_mm": 48.6,
    "culmen_depth_mm": 16.0,
    "flipper_length_mm": 230.0,
    "body_mass_g": 5800.0
}
```

And here is an example of the output we'd like to get from the endpoint:

```{json}
{
    "prediction": "Adelie",
    "confidence": 0.802672
}
```


### Step 1 - Creating the Preprocessing Script

The first component of our inference pipeline will transform the input data into the format the model expects.

We'll use the Scikit-Learn transformer we saved when we split and transformed the data. To deploy this component as part of an inference pipeline, we need to write a script that loads the transformer, uses it to modify the input data, and returns the output in the format the TensorFlow model expects.


We'll store the scripts of every component in a folder called `pipeline` and add it to the system path so we can later import it as a module.


Let's now create the script for the preprocessing component:


Let's test the script to ensure everything is working as expected:


### Step 2 - Creating the Postprocessing Script

The final component of our inference pipeline will transform the output from the model into a human-readable format.

We'll use the Scikit-Learn target transformer we saved when we split and transformed the data. To deploy this component as part of an inference pipeline, we need to write a script that loads the transformer, uses it to modify the output from the model, and returns a human-readable format.


Let's test the script to ensure everything is working as expected:


### Step 3 - Setting up the Inference Pipeline

We can now create a [PipelineModel](https://sagemaker.readthedocs.io/en/stable/api/inference/pipeline.html#sagemaker.pipeline.PipelineModel) to define our inference pipeline.

We'll use the model we generated in the Split and Transform step as the input to the first and last components of the inference pipeline. This `model.tar.gz` file contains the two transformers we need to preprocess and postprocess the data.


Let's create a variable with the URI to this file:


Here is the first component of the inference pipeline. It will preprocess the data before sending it to the TensorFlow model:


Here is the last component of the inference pipeline. It will postprocess the output from the TensorFlow model before sending it back to the user:


We can now create the inference pipeline using the three models:


### Step 4 - Configuring the Model Package Group

Let's define a new package group to register the Pipeline Model:


### Step 5 - Registering the Model

We'll modify the registration step to register the Pipeline Model in the Model Registry.


### Step 6 - Modifying the Deploy Step

Let's now modify the [LambdaStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.lambda_step.LambdaStep) to use the updated Registration Step.


### Step 7 - Modifying the Condition Step

Since we modified the Registration Step, we also need to modify the Condition Step to use the new registration:


### Step 8 - Creating the Pipeline

We can now define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


### Step 9 - Testing the Endpoint

Let's now test the endpoint. Notice that we can now send the raw data to the endpoint, and it will return the penguin's species in a human-readable format.


We can also test the endpoint by sending a JSON payload. Notice how you can use a deserealizer to automatically decode the response from the model.


And now let's send the same payload but return the prediction in CSV format:


## Session 15 - Custom Inference Script

This session creates a custom inference script to control the inference process in the SageMaker endpoint. This is an alternative to creating an inference pipeline to preprocess and postprocess the data that comes in and out of the model.


### Step 1 - Creating the Inference Script

Let's create a script where we'll manage the inference process in the endpoint.

We'll' include this code as part of the model assets to control the inference process on the SageMaker endpoint. SageMaker will automatically call the `handler()` function for every request to the endpoint. Check [How to implement the pre- and/or post-processing handler(s)](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/deploying_tensorflow_serving.html#how-to-implement-the-pre-and-or-post-processing-handler-s) for more information.


We can now create the script inside the folder.


Let's test the script to ensure everything is working as expected:


### Step 2 - Creating the Model

We can now create a new [TensorFlowModel](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-serving-model) including the `inference.py` file.

SageMaker triggers a repack operation whenever we specify the `source_dir` attribute in a model. We want that attribute to point to the local folder containing the `inference.py` file. SageMaker will automatically modify the original `model.tar.gz` package to include a `/code` folder containing the file.

Since we need access to Scikit-Learn in our script, we can include a `requirements.txt` file in the same location where the `inference.py` script is, and SageMaker will install everything in it.

To repack the model assets, SageMaker will automatically include a new step in the pipeline right before registering the model.

Here is what the new `model.tar.gz` package will look like:

```
model/
    |--[model_version_number]
        |--assets/
        |--variables/
        |--saved_model.pb
    |--features.joblib
    |--target.joblib
code/
    |--inference.py
    |--requirements.txt
```


Let's create a `requirements.txt` file with all the libraries we want SageMaker to install in the inference container.


We can now create the model using the `inference.py` script.


### Step 3 - Configuring the Model Package Group

Let's define a new group where we'll register the model using the custom `inference.py` script.


### Step 4 - Registering the Model

We can now modify the registration step to register the custom model in the Model Registry.


### Step 5 - Modifying the Deploy Step

Let's now modify the [LambdaStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.lambda_step.LambdaStep) to use the updated Registration Step.


### Step 6 - Modifying the Condition Step

Since we modified the Registration Step, we also need to modify the Condition Step to use the new registration:


### Step 7 - Creating the Pipeline

We can now define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


### Step 8 - Testing the Endpoint

Let's test the endpoint to make sure it works.


## Session 16 - Data Quality Baseline

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with a [Quality Check Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-quality-check) to compute a baseline for the data the endpoint expects.

This step will compute statistics and constraints from the data. We'll' later use this information as the baseline to detect data drift and other data quality issues.

<a href="images/data-quality-baseline.png" target="_blank"> <img src="images/data-quality-baseline.png" alt="Data Quality Baseline" style="max-width: 740px;" /></a>

Check [Monitor data quality](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-quality.html) for more information about monitoring data quality in SageMaker.


### Step 1 - Configuring Baseline Location

Let's start by defining the location where SageMaker will store the baseline data:


### Step 2 - Generating Data Quality Baseline

Let's configure a [QualityCheck Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-quality-check) to compute the general statistics of the data we used to build our model.

We can configure the instance that will run the quality check using the [CheckJobConfig](https://sagemaker.readthedocs.io/en/v2.73.0/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.check_job_config.CheckJobConfig) class, and we can use the `DataQualityCheckConfig` class to configure the job.

We are running this step with the following configuration:

-   `skip_check = True`: This parameter controls whether the step should skip checking the data against a previous baseline. Since we want to generate the baseline for the first time, we set it to `True`. After running the pipeline once to generate the baseline, we can set this parameter to `False` to ensure any new data follows the same distribution as the baseline.

-   `register_new_baseline = True`: This parameter controls whether the new calculated baseline will be registered in the Model Registry.

For more information about these configuration parameters, check [Baseline calculation and registration](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-quality-clarify-baseline-lifecycle.html).


### Step 3 - Setting up Model Metrics

We can configure a new set of [ModelMetrics](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_metrics.ModelMetrics) using the results of the Quality Step. Check [Baseline and model version lifecycle and evolution with SageMaker Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-quality-clarify-baseline-lifecycle.html#pipelines-quality-clarify-baseline-evolution) for an explanation of how SageMaker uses the `DriftCheckBaselines`.


### Step 4 - Registering the Model

Let's modify the registration step to use the new metrics and the drift baseline:


### Step 5 - Modifying the Condition Step

Since we modified the Registration Step, we also need to modify the Condition Step to use the new registration:


### Step 6 - Creating the Pipeline

We can now define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


### Step 7 - Checking Constraints and Statistics

Our pipeline generated data baseline statistics and constraints. We can take a look at what these values look like by downloading them from S3. You need to wait for the pipeline to finish running before these files are available.


Here are the data quality statistics:


Here are the data quality constraints:


## Session 17 - Model Quality Baseline

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with a [QualityCheck Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-quality-check) to compute a baseline for the model performance.

This step will compute the baseline metrics we will later use as the baseline to detect model drift.

To create a baseline to compare the model performance, we must create predictions for the test set and compare the model's metrics with the model performance on production data. We can do this by running a [Batch Transform Job](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) to predict every sample from the test set. We can use a [Transform Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-transform) as part of the pipeline to run this job.

<a href="images/model-quality-baseline.png" target="_blank"> <img src="images/model-quality-baseline.png" alt="Model Quality Baseline" style="max-width: 740px;" /></a>

Check [Monitor model quality](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality.html) for more information about monitoring model quality in SageMaker.


### Step 1 - Configuring Baseline Location

Let's start by defining the location where SageMaker will store the baseline data:


### Step 2 - Creating the Model

The [Transform Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-transform) requires a model to generate predictions, so we need a [Model Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-model) that creates a model:


### Step 3 - Setting up the Transform Step

We are going to use a [Batch Transform Job](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) to generate predictions for every sample from the test set.

This Batch Transform Job will run every sample from the training dataset through the model so we can compute the baseline metrics. Check [Run a Batch Transform Job](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#run-a-batch-transform-job) for more information about running a Batch Transform Job.


Let's start by configuring a [Transformer](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html) instance:


We can now set up the [Transform Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-transform) using the [Transformer](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html) we configured before.

Notice the following:

-   We'll generate predictions for the baseline test data that we generated when we split and transformed the data. This baseline is the same data we used to test the model, but it's in raw format.
-   The output of this Batch Transform Job will have two fields. The first one will be the ground truth label, and the second one will be the prediction of the model.


### Step 4 - Generating Model Quality Baseline

Let's now configure the [Quality Check Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-quality-check) and feed it the data we generated in the Transform Step. This step will automatically compute the performance metrics of the model on the test set.

We are running this step with the following configuration:

-   `skip_check = True`: This parameter controls whether the step should skip checking the data against a previous baseline. Since we want to generate the baseline for the first time, we set it to `True`. After running the pipeline once to generate the baseline, we can set this parameter to `False` to ensure any new data follows the same distribution as the baseline.

-   `register_new_baseline = True`: This parameter controls whether the new calculated baseline will be registered in the Model Registry.


### Step 5 - Setting up Model Metrics

We can configure a new set of [ModelMetrics](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_metrics.ModelMetrics) using the results of the Quality Step. Check [Baseline and model version lifecycle and evolution with SageMaker Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-quality-clarify-baseline-lifecycle.html#pipelines-quality-clarify-baseline-evolution) for an explanation of how SageMaker uses the `DriftCheckBaselines`.


### Step 6 - Registering the Model

Let's modify the registration step to use the new metrics and the drift baseline:


### Step 7 - Modifying the Condition Step

We need to modify the Condition Step to include the new Registration Step and the Transform and Quality Check Steps.


### Step 8 - Creating the Pipeline

We can now define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


### Step 9 - Checking Constraints

Our pipeline generated model baseline constraints. We can take a look at what these values look like by downloading them from S3. You need to wait for the pipeline to finish running before the file is available.


## Session 18 - Data Monitoring

This session creates a Monitoring Job to monitor the quality of the data received by the endpoint. This schedule will run periodically and check the data that goes into the endpoint against the baseline we generated before.

Check [Amazon SageMaker Model Monitor](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_monitoring.html) for an explanation of how to use SageMaker's Model Monitoring functionality. [Monitor models for data and model quality, bias, and explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html) is a much more extensive guide to monitoring in Amazon SageMaker.


### Step 1 - Deploying the Model

Let's deploy the latest approved model to an endpoint.

Since we need to do the same later, we can create a function to deploy the model. Notice how we need to enable Data Capture to monitor the data that goes in and out of the endpoint.


#| hide

<div class="alert" style="background-color:#0066cc;"><strong>Note:</strong> 
    The <code>%%script</code> cell magic is a convenient way to prevent the notebook from executing a specific cell. If you want to run the cell, comment out the line containing the <code>%%script</code> cell magic.
</div>


### Step 2 - Generating Fake Traffic

To test the monitoring functionality, we need to generate traffic to the endpoint. To generate traffic, we will send every sample from the dataset to the endpoint to simulate real prediction requests:


We can check the location where the endpoint stores the captured data, download a file, and display its content. It may take a few minutes for the first few files to show up in S3.

These files contain the data captured by the endpoint in a SageMaker-specific JSON-line format. Each inference request is captured in a single line in the `jsonl` file. The line contains both the input and output merged together:


### Step 3 - Creating Custom Preprocessing Script

SageMaker looks for violations in the data captured by the endpoint. By default, it combines the input data with the endpoint output and compares the result with the baseline we generated before. If we let SageMaker do this, we will get a few violations, for example an "extra column check" violation because the field `confidence` doesn't exist in the baseline data.

We can fix these violations by creating a preprocessing script configuring the data we want the monitoring job to use. Check [Preprocessing and Postprocessing](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-pre-and-post-processing.html) for more information about how to configure these scripts.

We'll store the script in a folder called `monitoring`:


We can now define the preprocessing script. Notice that this script will return a JSON object with a name for each feature and their value.


### Step 4 - Uploading Preprocessing Script

The monitoring schedule expects an S3 location pointing to the preprocessing script. Let's upload the script to the default bucket.


### Step 5 - Creating Monitoring Schedule

We can now set up the Data Quality Monitoring Job using the [DefaultModelMonitor](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_monitor.model_monitoring.DefaultModelMonitor) class.


Let's now create the monitoring schedule. Notice how we specify the `record_preprocessor_script` using the S3 location where we uploaded our script.

We are going to set up the monitoring schedule to run every hour. Keep in mind that SageMaker has a buffer period of 20 minutes to schedule an execution.


#| hide

<div class="alert" style="background-color:#0066cc;"><strong>Note:</strong> 
    The <code>%%script</code> cell magic is a convenient way to prevent the notebook from executing a specific cell. If you want to run the cell, comment out the line containing the <code>%%script</code> cell magic.
</div>


### Step 6 - Checking Violations

After the monitoring schedule runs for the first time, we can check the results of the last execution. If the job completed successfully, we can check if there are any violations.


### Step 7 - Deleting Monitoring Schedule

Once we are done with it, we can delete the Data Monitoring schedule.


## Session 19 - Model Monitoring

This session creates a Monitoring Job to monitor the quality of the model outputs. This schedule will run periodically and check the data that goes into the endpoint against the baseline we generated before.

Check [Amazon SageMaker Model Monitor](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_monitoring.html) for an explanation of how to use SageMaker's Model Monitoring functionality. [Monitor models for data and model quality, bias, and explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html) is a much more extensive guide to monitoring in Amazon SageMaker.


### Step 1 - Configuring Ground Truth Location

Let's start by defining the location where SageMaker will store the ground-truth generated by labeling the data received by the endpoint.


### Step 2 - Deploying the Model

Let's deploy the latest approved model to an endpoint.

Here, we can reuse the function we created before to deploy the model.


#| hide

<div class="alert" style="background-color:#0066cc;"><strong>Note:</strong> 
    The <code>%%script</code> cell magic is a convenient way to prevent the notebook from executing a specific cell. If you want to run the cell, comment out the line containing the <code>%%script</code> cell magic.
</div>


### Step 3 - Generating Fake Traffic

To test the monitoring functionality, we need to generate traffic to the endpoint. We can use the function we created before to generate fake traffic to the endpoint.


We can check the location where the endpoint stores the captured data, download a file, and display its content. It may take a few minutes for the first few files to show up in S3.

These files contain the data captured by the endpoint in a SageMaker-specific JSON-line format. Each inference request is captured in a single line in the `jsonl` file. The line contains both the input and output merged together:


### Step 4 - Generating Fake Labels

To test the performance of the model, we need to label the samples captured by the endpoint. We can simulate the labeling process by generating a random label for every sample. Check [Ingest Ground Truth Labels and Merge Them With Predictions](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-merge.html) for more information about this.


### Step 5 - Creating Monitoring Schedule

To set up a Model Quality Monitoring Job, we can use the [ModelQualityMonitor](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_monitor.model_monitoring.ModelQualityMonitor) class.

Check [Amazon SageMaker Model Quality Monitor](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_model_monitor/model_quality/model_quality_churn_sdk.html) for a complete tutorial on how to run a Model Monitoring Job in SageMaker.


Let's now create the monitoring schedule. The [EndpointInput](https://sagemaker.readthedocs.io/en/v2.24.2/api/inference/model_monitor.html#sagemaker.model_monitor.model_monitoring.EndpointInput) instance configures the attribute the monitoring job should use to determine the prediction from the model.

We are going to set up the monitoring schedule to run every hour. Keep in mind that SageMaker has a buffer period of 20 minutes to schedule an execution.


#| hide

<div class="alert" style="background-color:#0066cc;"><strong>Note:</strong> 
    The <code>%%script</code> cell magic is a convenient way to prevent the notebook from executing a specific cell. If you want to run the cell, comment out the line containing the <code>%%script</code> cell magic.
</div>


### Step 6 - Checking Violations

After the monitoring schedule runs for the first time, we can check the results of the last execution. If the job completed successfully, we can check if there are any violations.


### Step 7 - Deleting Monitoring Schedule

Once we are done with it, we can delete the Data Monitoring schedule.


## Session 20 - Shadow Deployments

This session configures an endpoint running a production and a shadow variant. Check [Safely validate models in production](https://docs.aws.amazon.com/sagemaker/latest/dg/model-validation.html) for more information.

<a href="images/shadow-deployment.png" target="_blank"> <img src="images/shadow-deployment.png" alt="Shadow Deployment" style="max-width: 740px;" /></a>


### Step 1 - Getting The Latest Models

We want to deploy the two latest approved models from the Model Registry to the same endpoint. The latest version of the model will act as the Shadow variant, and the previous version will act as the Production variant.


### Step 2 - Creating the Models

We want to deploy the two packages to a new endpoint. We'll use the boto3 API to deploy these models.

Let's start by creating the SageMaker Models.


Let's now create the Production model.


And now we can create the second model.


### Step 3 - Creating the Endpoint Configuration

We can now create the Endpoint Configuration using the two models


Let's define the location where SageMaker will output the information captured by the Shadow variant.


We can create the Endpoint Configuration now.


### Step 4 - Creating the Endpoint

Finally, we can create the Endpoint using the Endpoint Configuration we created before.


#| hide

<div class="alert" style="background-color:#0066cc;"><strong>Note:</strong> 
    The <code>%%script</code> cell magic is a convenient way to prevent the notebook from executing a specific cell. If you want to run the cell, comment out the line containing the <code>%%script</code> cell magic.
</div>


### Step 5 - Generating Traffic

Let's generate some traffic to the endpoint so we can test the Shadow variant.


### Step 6 - Checking Captured Data

Let's check the location where the endpoint stores the captured data, download a file, and display its content. It may take a few minutes for the first few files to show up in S3.

The endpoint will capture the data for both the Production and the Shadow variants.


### Step 7 - Deleting the Endpoint

Let's now delete the endpoint.


## Running the Pipeline

We can run any of the pipelines we defined before by enabling the cell below and specifying the pipeline we want to run.


#| hide

<div class="alert" style="background-color:#0066cc;"><strong>Note:</strong> 
    The <code>%%script</code> cell magic is a convenient way to prevent the notebook from executing a specific cell. If you want to run the cell, comment out the line containing the <code>%%script</code> cell magic.
</div>


## Deleting the Endpoint

After testing the endpoint, we need to ensure we delete it.
